In [ ]:
### Import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import os

In [ ]:
### Make sure that 'ggplot' style is used for all plots
plt.style.use('ggplot')
# plt.style.available ### To view all other available styles

In [ ]:
### Set Working Directory (WD)
os.chdir('/Volumes/GoogleDrive/My Drive/CEMEX/Data Translators/GitHub/rgamerosl/capstone-project')

In [ ]:
### Read the data
df = pd.read_excel("dataset/data_v0.xlsx")

In [ ]:
df.info()

In [ ]:
### For some strange reason when reading the Excel file it detects 1'021,336 entries. However I know there are only 466,786 differente entries, eveything else are just empty rows
df = df.iloc[0:466786,0:18]
df.info()

In [ ]:
df_backup = df.copy(deep=True)

In [ ]:
display(df.head(7))

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
### First replace empty strings with nan
df['liters_per_hour'] = df['liters_per_hour'].replace(r'^\s*$', np.nan, regex=True)
### Transform strings into float numbers
df['liters_per_hour'] = df['liters_per_hour'].astype(float)

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
plt.hist(df['liters_per_hour'])
plt.show()

In [ ]:
### Everything bigger than 20 should be nan
df.loc[abs(df['liters_per_hour']) > 20,'liters_per_hour'] = np.nan
### Also every negative value should be nan
df.loc[df['liters_per_hour'] < 0,'liters_per_hour'] = np.nan

In [ ]:
plt.hist(df['liters_per_hour'])
plt.show()

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
### Amount of nan in the liters_per_hour (target value)
df['liters_per_hour'].isna().sum()/len(df['liters_per_hour'])